In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip -q "./drive/My Drive/Colab Notebooks/DL_Project/Dataset.zip"

In [0]:
import os

os.getcwd()
path = 'Dataset/'
train_image_path = path + 'images/images_train/'
validation_image_path = path + 'images/images_validation/'
test_image_path = path + 'images/images_test/'
train_formula_path = path + 'formulas/train_formulas.txt'
validation_formula_path = path + 'formulas/validation_formulas.txt'


In [0]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

n_train, n_validation, n_test = 66509, 7391, 8250

images_train = []
images_validation = []
images_test = []

for i in range(n_train):
  img = np.asarray(Image.open(train_image_path + '{}.png'.format(i)))
  images_train.append(img)

for i in range(n_validation):
  img = np.asarray(Image.open(validation_image_path + '{}.png'.format(i)))
  images_validation.append(img)

for i in range(n_test):
  img = np.asarray(Image.open(test_image_path + '{}.png'.format(i)))
  images_test.append(img)

In [0]:
images_train = np.array(images_train)
images_validation = np.array(images_validation)
images_test = np.array(images_test)

In [0]:
images_train = images_train.reshape(n_train, 1, 60, 400)
images_validation = images_validation.reshape(n_validation, 1, 60, 400)
images_test = images_test.reshape(n_test, 1, 60, 400)

In [0]:
train_formulas = open(train_formula_path, 'r').readlines()
validation_formulas = open(validation_formula_path, 'r').readlines()
bos_token = '<BOS>'
eos_token = '<EOS>'
pad_token = '<PAD>'
unk_token = '<UNK>'

MAX_LENGTH = 0

for i in range(n_train):
  train_formulas[i] = [bos_token] + train_formulas[i].split() + [eos_token]
  MAX_LENGTH = max(MAX_LENGTH, len(train_formulas[i]))

for i in range(n_validation):
  validation_formulas[i] = [bos_token] + validation_formulas[i].split() + [eos_token]
  MAX_LENGTH = max(MAX_LENGTH, len(validation_formulas[i]))

train_formula_lengths = np.zeros((n_train,))
validation_formula_lengths = np.zeros((n_validation,))
# print(MAX_LENGTH)

for i in range(n_train):
  length = len(train_formulas[i])
  train_formula_lengths[i] = length
  train_formulas[i] += [pad_token for _ in range(MAX_LENGTH - length)]

for i in range(n_validation):
  length = len(validation_formulas[i])
  validation_formula_lengths[i] = length
  validation_formulas[i] += [pad_token for _ in range(MAX_LENGTH - length)]


# print(len(train_formulas))
# print(len(validation_formulas))

# print(train_formulas[0])

In [0]:
train_formula_lengths = train_formula_lengths.astype(int)
validation_formula_lengths = validation_formula_lengths.astype(int)

In [0]:
all_tokens = [unk_token]
for i in range(n_train):
  for j in range(len(train_formulas[i])):
    if train_formulas[i][j] not in all_tokens:
      all_tokens.append(train_formulas[i][j])
#   all_tokens += train_formulas[i]

# all_tokens = list(set(all_tokens))
num_tokens = len(all_tokens)
# print(num_tokens)
# print(all_tokens)
# print('\\over' in all_tokens)
# print('\\' in all_tokens)
token_idx = {}
for i in range(num_tokens):
  token_idx[all_tokens[i]] = i
# print(token_idx['<BOS>'])
# print(token_idx[bos_token])
# print(token_idx['{'])

In [10]:
print(all_tokens[34])
print(token_idx['\\bigg'])
print(len(all_tokens))

\bigg
34
566


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random

In [0]:
for i in range(n_train):
  for j in range(MAX_LENGTH):
    train_formulas[i][j] = token_idx[train_formulas[i][j]]

for i in range(n_validation):
  for j in range(MAX_LENGTH):
    validation_formulas[i][j] = token_idx.get(validation_formulas[i][j], token_idx[unk_token])

train_formulas = torch.LongTensor(train_formulas)
validation_formulas = torch.LongTensor(validation_formulas)

# print(train_formulas.shape)
# print(validation_formulas.shape)

In [0]:
CUDA = True
BATCH_SIZE = 20

In [0]:
class MyReshape(nn.Module):
  def forward(self, input):
    return input.view(input.size(0), input.size(1)*input.size(2), input.size(3)).transpose(1,2)


cnn_encoder = nn.Sequential(
  nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
  nn.ReLU(),
  
  nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0)),
  
  nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
  nn.ReLU(),
  
  nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0)),
  
  nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
  nn.BatchNorm2d(256),
  nn.ReLU(),
  
  nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
  nn.ReLU(),
  
  nn.MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=(0, 0)),
  
  nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
  nn.BatchNorm2d(512),
  nn.ReLU(),
  
  nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=(0, 0)),
  
  nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
  nn.BatchNorm2d(512),
  nn.ReLU(),
  
  MyReshape()
    
).cuda()

In [15]:
dummy_v = cnn_encoder(torch.Tensor(images_validation[0:10]).cuda())
print(dummy_v.shape)
_, seq_len, feature_num = dummy_v.shape
encoder_hidden_size = 256
num_layers = 1

lstm_encoder = nn.LSTM(feature_num, encoder_hidden_size, num_layers, bidirectional=True, batch_first=True).cuda()

torch.Size([10, 50, 3584])


In [0]:
v = cnn_encoder(torch.Tensor(images_validation[0:10]).cuda())
# print(v.shape)
# print(lstm_encoder.device)
v_tilda, (hn, cn) = lstm_encoder(v)
# print(v_tilda.shape)

In [0]:
class AttnDecoder(nn.Module):
  def __init__(self, num_tokens, embedding_dim, hidden_dim, output_dim, attn_dim):
    super(AttnDecoder, self).__init__()
    self.embedding_dim = embedding_dim
    self.hidden_dim = hidden_dim
    self.output_dim = output_dim
    self.attn_dim = attn_dim # dimension e Wh o Wv o Beta
    
    self.embedding = nn.Embedding(num_tokens, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim+output_dim, hidden_dim, 1, batch_first=True)
    self.Wh = nn.Linear(hidden_dim, attn_dim)
    self.Wv = nn.Linear(hidden_dim, attn_dim)
    self.tanh = nn.Tanh()
    self.beta = nn.Linear(attn_dim,1)
    self.Wc1 = nn.Linear(hidden_dim, output_dim)
    self.Wc2 = nn.Linear(hidden_dim, output_dim)
    self.o_tanh = nn.Tanh()
    self.Wout = nn.Linear(output_dim, num_tokens)
    self.nll = nn.NLLLoss(reduction='sum')
    
    
  def forward(self, last_y, last_o, last_h, last_c_memory, v, ):
    
    linear_trans = self.Wh(last_h).unsqueeze(1) + self.Wv(v) # Wh * h_t-1 + Wv * v_t
    e_i = self.beta(self.tanh(linear_trans)).squeeze(2) # beta_T * tanh(linear_trans)
    alpha_i = F.softmax(e_i, dim=1)
    c_i = (alpha_i.unsqueeze(2) * v).sum(dim=1)
    
    embedded = self.embedding(last_y)
    ltsm_input = torch.cat((embedded, last_o), dim=1)
    ltsm_input = ltsm_input.unsqueeze(1)
    h_t, (_, c_memory_t) = self.lstm(ltsm_input, (last_h.unsqueeze(0), last_c_memory.unsqueeze(0)))
    h_t = h_t.squeeze(1)
    c_memory_t = c_memory_t.squeeze(0)
    
    o_t = self.o_tanh(self.Wc1(h_t) + self.Wc2(c_i))
    prob = F.log_softmax(self.Wout(o_t), dim=1)
    
    return prob, o_t, h_t, c_memory_t,
    
  def train(self, v_tilda, sequence, max_len, teacher_forcing_ratio=1):
    batch_num = v_tilda.shape[0]
    h_0 = v_tilda[:,-1,:].contiguous()
    c_0 = torch.zeros_like(h_0).cuda()
    o_0 = torch.zeros(batch_num, self.output_dim).cuda()
    y_0 = sequence[:, 0].cuda()
    
    loss = 0
    for i in range(max_len-1):
#       y_0 = sequence[:, i].cuda()
      prob, o_0, h_0, c_0 = self.forward(y_0, o_0, h_0, c_0, v_tilda)
      loss += self.nll(prob, sequence[:, i+1].cuda())
      
      y_0 = sequence[:, i+1]
#       eps = random.random()
#       if eps > teacher_forcing_ratio:
#         y_0 = torch.distributions.Categorical(prob).sample()
#       else:
#         y_0 = sequence[:, i+1]
      
    return loss
  
      

attn_decoder = AttnDecoder(num_tokens, embedding_dim=80, hidden_dim=512, output_dim=600, attn_dim=512).cuda()
    

In [0]:
def predict(image, cnn_encoder, lstm_encoder, attn_decoder, max_len=178):
  v = cnn_encoder(image.unsqueeze(0))
  v_tilda, _ = lstm_encoder(v)
  
  h_0 = v_tilda[0, -1, :].unsqueeze(0)
  c_0 = torch.zeros_like(h_0).cuda()
  o_0 = torch.zeros(attn_decoder.output_dim).unsqueeze(0).cuda()
  
  sequence = [token_idx[bos_token]]
  
  for i in range(max_len-1):
    y_0 = torch.LongTensor([sequence[-1]]).cuda()
    prob, o_0, h_0, c_0 = attn_decoder(y_0, o_0, h_0, c_0, v_tilda)
    prob = prob.squeeze(0)
    max_idx = torch.argmax(prob).item()
#     print(2.71 ** prob[max_idx].item())
    sequence.append(max_idx)
    if max_idx == token_idx[eos_token] or max_idx == token_idx[pad_token]:
      return sequence
    
  return sequence
      
  

In [25]:
index = 1
test_image = torch.Tensor(images_validation[index]).cuda()
test_image -= 128
test_image /= 128
formula = predict(test_image, cnn_encoder, lstm_encoder, attn_decoder)
for i in range(len(formula)):
  formula[i] = all_tokens[formula[i]]
# print(formula)
sentence = ""
for i in range(1, len(formula)-1):
  sentence += formula[i]
  sentence += " "
print(sentence)
# formula = ''.join(formula[1:-1], )
# print(formula)

# plt.imshow(images_validation[index].reshape(60,400))
# plt.show()
# print(images_train[0])

D _ { \mu } = \partial _ { \mu } + A _ { \mu } , \qquad A _ { \mu } = i g A ^ { a } _ { \mu } ( x ) T ^ { a } 


In [20]:
last_y = torch.LongTensor([0,1,2,3,4,5,6,7,8,9])
# print(attn_decoder.embedding(last_y))
# kkk = attn_decoder.embedding(last_y)
# print(kkk.shape)
# print(torch.cat((kkk, torch.randn(10, 564)), dim=1).shape)
attn_decoder(last_y.cuda(), torch.randn(10, 600).cuda(), torch.randn(10,512).cuda(), torch.randn(10,512).cuda(), torch.randn(10,50,512).cuda())
# print(type(train_formulas))
print(attn_decoder.train(torch.randn(20,50,512).cuda(), train_formulas[0:20].cuda(), max(train_formula_lengths[0:20])))
# print(attn_decoder.train(torch.randn(20,50,512), train_formulas[0:20])[0].shape)

# attn_decoder.test(torch.randn(20, 50, 512).cuda())
print()

tensor(13784.0469, device='cuda:0', grad_fn=<AddBackward0>)



In [0]:
def make_batches():
  counter = 0
  while True:
    start = counter * BATCH_SIZE
    end = min((counter+1) * BATCH_SIZE, n_train)
    if end == n_train:
      counter = 0
    x = torch.Tensor(images_train[start:end]).cuda()
    y = train_formulas[start:end].cuda()
    max_len = max(train_formula_lengths[start:end])
    counter += 1
    x -= 128
    x /= 128
    
    yield x, y, max_len


In [0]:
batch_maker = make_batches()
# print(train_formulas[0])
print(next(batch_maker)[0])

In [0]:
attn_decoder_params = list(attn_decoder.parameters())
print(attn_decoder.parameters)

In [0]:
teacher_forcing = 1
N_EPOCH = 12
lr = 0.001

params = list(cnn_encoder.parameters()) + list(lstm_encoder.parameters()) + list(attn_decoder.parameters())
optimizer = optim.Adam(params, lr=lr)

In [23]:
for i in range(N_EPOCH):
  batch_maker = make_batches()
  epoch_loss, cum_loss = 0, 0
  for j in range(int(n_train/BATCH_SIZE)+1):
    x, y, max_len = next(batch_maker)
    
    cnn_encoded = cnn_encoder(x)
    v_tilda, _ = lstm_encoder(cnn_encoded)
    loss = attn_decoder.train(v_tilda, y, max_len, teacher_forcing_ratio=teacher_forcing)
    cum_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if j % 100 == 0 and j != 0:
      print("processes until data number {}. cumulative loss for {} items is: {}".format(j*BATCH_SIZE, 100*BATCH_SIZE, cum_loss/(100*BATCH_SIZE)))
      epoch_loss += cum_loss
      cum_loss = 0
  print("----------------------------")
  print("epoch {} cumulative loss = {}".format(i+1, epoch_loss/n_train))
  print("----------------------------")
  teacher_forcing -= 0.1
  torch.save(cnn_encoder.state_dict(), '/content/drive/My Drive/Colab Notebooks/DL_Project/Saved_Models/cnn_train_0_epoch{}.pth'.format(i))
  torch.save(lstm_encoder.state_dict(), '/content/drive/My Drive/Colab Notebooks/DL_Project/Saved_Models/lstm_train_0_epoch{}.pth'.format(i))
  torch.save(attn_decoder.state_dict(), '/content/drive/My Drive/Colab Notebooks/DL_Project/Saved_Models/decoder_train_0_epoch{}.pth'.format(i))
  


processes until data number 2000. cumulative loss for 2000 items is: 196.71745239257814
processes until data number 4000. cumulative loss for 2000 items is: 149.11324182128905
processes until data number 6000. cumulative loss for 2000 items is: 136.63633837890626
processes until data number 8000. cumulative loss for 2000 items is: 132.33811865234375
processes until data number 10000. cumulative loss for 2000 items is: 125.1440336303711
processes until data number 12000. cumulative loss for 2000 items is: 124.79572412109376
processes until data number 14000. cumulative loss for 2000 items is: 118.67991687011718
processes until data number 16000. cumulative loss for 2000 items is: 115.84001318359375
processes until data number 18000. cumulative loss for 2000 items is: 114.63165368652344
processes until data number 20000. cumulative loss for 2000 items is: 112.67596661376953
processes until data number 22000. cumulative loss for 2000 items is: 110.97641668701172
processes until data numbe

In [0]:
torch.save(cnn_encoder.state_dict(), '/content/drive/My Drive/Colab Notebooks/DL_Project/Saved_Models/new/cnn.pth')
# torch.save(lstm_encoder.state_dict(), '/content/drive/My Drive/Colab Notebooks/DL_Project/Saved_Models/lstm.pth')
# torch.save(attn_decoder.state_dict(), '/content/drive/My Drive/Colab Notebooks/DL_Project/Saved_Models/decoder.pth')

      

In [44]:
cnn_encoder_loaded = nn.Sequential(
  nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
  nn.ReLU(),
  
  nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0)),
  
  nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
  nn.ReLU(),
  
  nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0)),
  
  nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
  nn.BatchNorm2d(256),
  nn.ReLU(),
  
  nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
  nn.ReLU(),
  
  nn.MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=(0, 0)),
  
  nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
  nn.BatchNorm2d(512),
  nn.ReLU(),
  
  nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=(0, 0)),
  
  nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
  nn.BatchNorm2d(512),
  nn.ReLU(),
  
  MyReshape()
    
)
cnn_encoder_loaded.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/DL_Project/Saved_Models/cnn_train_0_epoch6.pth'))
cnn_encoder_loaded.cuda()
cnn_encoder_loaded.eval()

lstm_encoder_loaded = nn.LSTM(feature_num, encoder_hidden_size, num_layers, bidirectional=True, batch_first=True)
lstm_encoder_loaded.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/DL_Project/Saved_Models/lstm_train_0_epoch6.pth'))
lstm_encoder_loaded.cuda()
lstm_encoder_loaded.eval()

attn_decoder_loaded = AttnDecoder(num_tokens, embedding_dim=80, hidden_dim=512, output_dim=600, attn_dim=512)
attn_decoder_loaded.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/DL_Project/Saved_Models/decoder_train_0_epoch6.pth'))
attn_decoder_loaded.cuda()
# attn_decoder_loaded.eval()



AttnDecoder(
  (embedding): Embedding(566, 80)
  (lstm): LSTM(680, 512, batch_first=True)
  (Wh): Linear(in_features=512, out_features=512, bias=True)
  (Wv): Linear(in_features=512, out_features=512, bias=True)
  (tanh): Tanh()
  (beta): Linear(in_features=512, out_features=1, bias=True)
  (Wc1): Linear(in_features=512, out_features=600, bias=True)
  (Wc2): Linear(in_features=512, out_features=600, bias=True)
  (o_tanh): Tanh()
  (Wout): Linear(in_features=600, out_features=566, bias=True)
  (nll): NLLLoss()
)

In [45]:
index = 0
validation_preds = ""
for index in range(n_test):
  test_image = torch.Tensor(images_test[index]).cuda()
  test_image -= 128
  test_image /= 128
  formula = predict(test_image, cnn_encoder_loaded, lstm_encoder_loaded, attn_decoder_loaded)
  for i in range(len(formula)):
    formula[i] = all_tokens[formula[i]]
  # print(formula)
  sentence = ""
  for i in range(1, len(formula)-1):
    sentence += formula[i]
    sentence += " "
  validation_preds += sentence
  validation_preds += "\n"
val_preds = open('test_formulas_prediction.txt', 'w')
val_preds.write(validation_preds)
# print(sentence)
# formula = ''.join(formula[1:-1], )
# print(formula)

# plt.imshow(images_validation[index].reshape(60,400))
# plt.show()
# print(images_train[0])

1049105

In [39]:
import numpy as np
from pylatex import Document, Section, Subsection, Table, Math, TikZ, Axis, Plot, Figure, Package
from pylatex.numpy import Matrix
from pylatex.utils import italic, escape_latex


ModuleNotFoundError: ignored

In [0]:
import pylatex.